In [ ]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
from transformers import RobertaTokenizer, RobertaConfig, RobertaModel
from transformers import InputExample, InputFeatures
import tensorflow as tf
import pandas as pd
import re

In [2]:
df = pd.read_csv("/kaggle/input/c-code-from-project-codenet/function_swap_samples_revised.txt",sep='\t')#/kaggle/input/my-data
#df.columns = ["function_name","arg1","arg2","arg_type","param1","param2","labels"]

In [3]:
df

,function_name,arg1,arg2,arg_type,param1,param2,labels
0,fopen,file,r,FILE *,__filename,__modes,0
1,fopen,r,file,FILE *,__filename,__modes,1
2,fopen,tempbuf,wt,FILE *,__filename,__modes,0
3,fopen,wt,tempbuf,FILE *,__filename,__modes,1
4,strcmp,ext,extension,int,NaN,NaN,0
...,...,...,...,...,...,...,...
87685,test3f,-,1.0f,float,x,y,1
87686,test3f,2.0f,1.0f,float,x,y,0
87687,test3f,1.0f,2.0f,float,x,y,1
87688,test3f,2.0f,-,float,x,y,0


In [4]:
df.isna().sum()

function_name      646
arg1               702
arg2               702
arg_type             0
param1           64808
param2           64778
labels               0
dtype: int64

In [5]:
# df.drop(['Unnamed: 7','Unnamed: 8','Unnamed: 9','Unnamed: 10','Unnamed: 11'],axis=1,inplace=True)
# df

In [6]:
# df.isna().sum()

In [7]:
df["function_name"] = df["function_name"].fillna('[UNK]')
df["arg1"]=df["arg1"].fillna('[UNK]')
df["arg2"]=df["arg2"].fillna('[UNK]')
df["arg_type"]=df["arg_type"].fillna('[UNK]')
df["param1"]=df["param1"].fillna('[UNK]')
df["param2"]=df["param2"].fillna('[UNK]')

In [8]:
df.isna().sum()

function_name    0
arg1             0
arg2             0
arg_type         0
param1           0
param2           0
labels           0
dtype: int64

In [9]:
# df.dropna(inplace=True)

In [10]:
df.isna().sum()

function_name    0
arg1             0
arg2             0
arg_type         0
param1           0
param2           0
labels           0
dtype: int64

In [11]:
df['total'] = df["function_name"] + '[PAD]'+ df["arg1"] + '[PAD]'+df["arg2"] + '[PAD]'+ df["arg_type"] + '[PAD]'+df["param1"]+ '[PAD]'+ df["param2"]

In [12]:
df.drop(["function_name","arg1","arg2","arg_type","param1","param2"],axis=1,inplace=True)

In [13]:
df = df.sample(frac = 1) # shuffling the dataset
df.reset_index(drop=True,inplace=True)

In [14]:
df.shape

(87690, 2)

In [15]:
df.drop_duplicates(inplace=True)

In [16]:
df.shape

(49788, 2)

In [18]:
df

,labels,total
0,1,outb[PAD]0x370[PAD]0xaa[PAD]int[PAD][UNK][PAD]...
1,0,fopen[PAD]dbspecs_f[PAD]r[PAD]FILE *[PAD]__fil...
2,1,write_io_cr[PAD]new_units[PAD]0xF1[PAD]void[PA...
3,1,__builtin_expect[PAD]1[PAD]([PAD]long[PAD][UNK...
4,1,queue_work[PAD]&[PAD]priv[PAD]int[PAD][UNK][PA...
...,...,...
87675,1,ktime_add[PAD]req[PAD]op_metrics[PAD]int[PAD][...
87681,1,VS[PAD]([PAD]_MEIPASS2 is %s\n[PAD]int[PAD][UN...
87682,1,strcmp[PAD]tcp[PAD]tokennet[PAD]int[PAD]__s1[P...
87683,1,strcmp[PAD]file:test[PAD]operation[PAD]int[PAD...


In [19]:
df.shape

(49788, 2)

In [28]:
 from transformers import AutoTokenizer, AutoModel
from transformers import RobertaTokenizer, T5ForConditionalGenerationcon

ImportError: cannot import name 'T5ForConditionalGenerationcon' from 'transformers' (/opt/conda/lib/python3.7/site-packages/transformers/__init__.py)

In [27]:
#classifier = TFBertForSequenceClassification.from_pretrained("codebert-base")
# classifier = AutoModel.from_pretrained("microsoft/codebert-base")

In [22]:
tokenizer = RobertaTokenizer.from_pretrained('Salesforce/codet5-base')
classifier= T5ForConditionalGeneration.from_pretrained('Salesforce/codet5-base'

Downloading:   0%|          | 0.00/1.53k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/850M [00:00<?, ?B/s]

In [23]:
#tokenizer = BertTokenizer.from_pretrained("codebert-base")
# tokenizer = AutoTokenizer.from_pretrained("microsoft/codebert-base")

In [25]:
classifier.from_config(config)

AttributeError: 'T5ForConditionalGeneration' object has no attribute 'from_config'

In [ ]:
m = len(df)
sample = int(m/10)

In [ ]:
train_df = df[sample:]
test_df = df[:sample]

In [ ]:
# InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this case
#                                                           text_a = x[code_snippet], 
#                                                           text_b = None,
#                                                           label = x[label_of_code_snippet])

In [ ]:
def convert_data_to_examples(train, test, code_snippet, label_of_code_snippet): 
  train_InputExamples = train.apply(lambda x: InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this case
                                                          text_a = x[code_snippet], 
                                                          text_b = None,
                                                          label = x[label_of_code_snippet]), axis = 1)

  validation_InputExamples = test.apply(lambda x: InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this case
                                                          text_a = x[code_snippet], 
                                                          text_b = None,
                                                          label = x[label_of_code_snippet]), axis = 1)
  
  return train_InputExamples, validation_InputExamples

In [ ]:
train_InputExamples, validation_InputExamples = convert_data_to_examples(train_df, 
                                                                           test_df, 
                                                                           'total', 
                                                                           'labels')

In [ ]:
train_df

In [ ]:
test_df

In [ ]:
def convert_examples_to_tf_dataset(examples, tokenizer, max_length=128):
    features = [] # -> will hold InputFeatures to be converted later

    for e in examples:
        # Documentation is really strong for this method, so please take a look at it
        input_dict = tokenizer.encode_plus(
            e.text_a,
            add_special_tokens=True,
            max_length=max_length, # truncates if len(s) > max_length
            return_token_type_ids=True,
            return_attention_mask=True,
            pad_to_max_length=True, # pads to the right by default # CHECK THIS for pad_to_max_length
            truncation=True
        )

        input_ids, token_type_ids, attention_mask = (input_dict["input_ids"],
            input_dict["token_type_ids"], input_dict['attention_mask'])

        features.append(
            InputFeatures(
                input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids, label=e.label
            )
        )

    def gen():
        for f in features:
            yield (
                {
                    "input_ids": f.input_ids,
                    "attention_mask": f.attention_mask,
                    "token_type_ids": f.token_type_ids,
                },
                f.label,
            )

    return tf.data.Dataset.from_generator(
        gen,
        ({"input_ids": tf.int32, "attention_mask": tf.int32, "token_type_ids": tf.int32}, tf.int64),
        (
            {
                "input_ids": tf.TensorShape([None]),
                "attention_mask": tf.TensorShape([None]),
                "token_type_ids": tf.TensorShape([None]),
            },
            tf.TensorShape([]),
        ),
    )

In [ ]:
code_snippet = 'total'
label_of_code_snippet = 'labels'

In [ ]:
train_InputExamples, validation_InputExamples = convert_data_to_examples(train_df, test_df, code_snippet, label_of_code_snippet)

train_data = convert_examples_to_tf_dataset(list(train_InputExamples), tokenizer)
train_data = train_data.shuffle(100).batch(32).repeat(2)

validation_data = convert_examples_to_tf_dataset(list(validation_InputExamples), tokenizer)
validation_data = validation_data.batch(32)

In [ ]:
classifier.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0), 
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
              metrics=[tf.keras.metrics.SparseCategoricalAccuracy('accuracy')])

In [ ]:
history = classifier.fit(train_data, epochs=20, validation_data=validation_data,verbose=1,batch_size=256,steps_per_epoch=175)

In [ ]:
import keras
from matplotlib import pyplot as plt

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
my_predict = classifier.predict(test_df)

In [ ]:
param_grid = {
    "per_gpu_batch_size": [16, 32,64],
    "learning_rate": [5e-6, 1e-5, 5e-5],
    "num_epochs": [5, 10, 25,40]
    "steps_per_epoch":[2128,1064,425,266]
}

In [ ]:
from sklearn import model_selection

In [ ]:
model = model_selection.GridSearchCV(
    estimator=classifier,
    param_grid=param_grid,
    scoring="accuracy",
    verbose=1,
    n_jobs=2,
    cv=5
)

In [ ]:
model.fit(train_data)